In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

np.random.seed(0)

In [25]:
vgg_complete.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [37]:
size = 224
training_path = 'C:\\Users\\Mark\\Desktop\\Images\\Training'
test_path = 'C:\\Users\\Mark\Desktop\\Images\\Test'

vgg_model = VGG16(weights = 'imagenet', include_top = False, input_shape=(size,size,3))

x = Flatten()(vgg_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

vgg_complete = Model(vgg_model.input, predictions)


vgg_complete.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

58892288/58889256 [==============================] - 11s 0us/step


In [38]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(training_path,
target_size = (size,size),
batch_size = 16,
class_mode = 'binary',
shuffle = True)

test_set = test_datagen.flow_from_directory(test_path,
target_size = (size,size),
batch_size = 16,
class_mode = 'binary')

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [39]:
vgg_complete.fit_generator(training_set,
steps_per_epoch = 50,
epochs = 3,
validation_data = test_set,
validation_steps = 800 // 16)

Epoch 1/3
 4/50 [=>............................] - ETA: 11:09 - loss: 7.5794 - accuracy: 0.4688

KeyboardInterrupt: 

In [0]:
# test the NN on a single image
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('gdrive/My Drive/AppliedProject/NewData/Test/Fake/easy_201_0001.jpg', target_size = (size,size))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(result)
if result[0][0] == 1:
  prediction = 'fake'
else:
  prediction = 'real'
  
prediction